In [ ]:
!pip3 install transformers torch scikit-learn accelerate emoji==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49719 sha256=b68803626db297459044d66ddacdf3108077f41827fbf805e57997b5126b2f3e
  Stored in directory: /root/.cache/pip/wheels/0d/bf/a2/536017b4a6232aef0fb92831af35facd6590c0af0f3983f63b
Successfully built emoji


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/machineLearning/twitter-datasets.zip" "/content"
!unzip -q twitter-datasets.zip
print("Data ready")

Data ready


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, set_seed, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import csv
from os import path
import random

fixing the seeds for repetability

In [ ]:
random.seed(42)
np.random.seed(42)
set_seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

Useful methods

In [ ]:
def load_tweets(filename):
  with open(filename, 'r', encoding='utf-8') as f:
    return [line.strip() for line in f]

In [ ]:
def create_csv_submission(ids, y_pred, file_name):
    """
    This function creates a csv file named `file_name` in the format required for a submission in Kaggle or AIcrowd.
    The file will contain two columns the first with `ids` and the second with `y_pred`.
    y_pred must be a list or np.array of 1 and -1 otherwise the function will raise a ValueError.

    Args:
        ids (list,np.array): indices
        y_pred (list,np.array): predictions on data correspondent to indices
        name (str): name of the file to be created
    """
    # Check that y_pred only contains -1 and 1
    if not all(i in [-1, 1] for i in y_pred):
        raise ValueError("y_pred can only contain values -1, 1")

    with open(file_name, "w") as csv_file:
        fieldnames = ["Id", "Prediction"]
        writer = csv.DictWriter(csv_file, delimiter = ",", fieldnames=fieldnames)
        writer.writeheader()
        for i in range(len(ids)):
            writer.writerow({"Id": int(ids[i]), "Prediction": int(y_pred[i])})

Data Loading

In [ ]:
tweet_pos = load_tweets("twitter-datasets/train_pos.txt")
tweet_neg = load_tweets("twitter-datasets/train_neg.txt")
tweet_test = load_tweets("twitter-datasets/test_data.txt")

tweets = tweet_pos + tweet_neg
labels = [1]*len(tweet_pos) + [0]*len(tweet_neg)

Preparing the data

In [ ]:
class TwitterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = dict(encodings) # Conversion en dict pour éviter le KeyError
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_tweets, eval_tweets, train_labels, eval_labels = train_test_split(tweets, labels, test_size=0.2, random_state=42, stratify=labels)

model_name = "vinai/bertweet-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenizing train data
train_encodings = tokenizer(train_tweets, truncation=True, padding=True, max_length=128)

# Tokenizing test data
eval_encodings = tokenizer(eval_tweets, truncation=True, padding=True, max_length=128)

train_dataset = TwitterDataset(train_encodings, train_labels)
eval_dataset = TwitterDataset(eval_encodings, eval_labels)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Setting up the model

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def compute_metrics(pred):
    labels = pred.label_ids
    acc = accuracy_score(labels, pred.predictions.argmax(-1))
    f1 = f1_score(labels, pred.predictions.argmax(-1))
    return {"accuracy": acc, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    lr_scheduler_type="cosine",
    learning_rate=1e-5,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],
)

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Training the model

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dejeanadrien (dejeanadrien-epfl) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.330200,0.294573,0.880575,0.885835
1000,0.272900,0.253844,0.895325,0.896778
1500,0.260600,0.262051,0.893650,0.891158
2000,0.263200,0.241889,0.899700,0.901124
2500,0.244700,0.241150,0.902175,0.904605
3000,0.237400,0.233837,0.902950,0.902955
3500,0.234700,0.238650,0.901750,0.900127
4000,0.245400,0.234260,0.902875,0.902487
4500,0.233000,0.225321,0.906800,0.907190
5000,0.248300,0.225120,0.906325,0.906714


TrainOutput(global_step=14000, training_loss=0.2108228155544826, metrics={'train_runtime': 1776.3519, 'train_samples_per_second': 450.361, 'train_steps_per_second': 14.074, 'total_flos': 1.680621866112e+16, 'train_loss': 0.2108228155544826, 'epoch': 2.8})

Doing the final prediction

In [ ]:
final_encoding = tokenizer(tweet_test, truncation=True, padding=True, max_length=128)
final_dataset = TwitterDataset(final_encoding, [0]*len(tweet_test))

predictions = trainer.predict(final_dataset)
final_proba_predictions = predictions.predictions.argmax(-1)

final_predictions = [1 if i == 1 else -1 for i in final_proba_predictions]

create_csv_submission(range(1, len(final_predictions)+1), final_predictions, "submission.csv")

Downloading the submission file

In [ ]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>